单页

In [3]:
import requests
from lxml import etree
import re

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"}
url = 'https://movie.douban.com/top250?start=0'
response = requests.get(url, headers=headers)
tree = etree.HTML(response.content.decode('utf-8'))

In [65]:
f = open('豆瓣top250_1.csv', 'w', encoding='utf-8')

for info in tree.xpath('//div[@class="info"]'):
    title = info.xpath('./div[@class="hd"]/a/span[@class="title"]/text()')[0]
    brief = ' '.join([line.strip().replace('\xa0', ' ') for line in info.xpath('./div[@class="bd"]/p/text()')]).strip()
    rating_num = info.xpath('./div[@class="bd"]/div[@class="star"]/span[@class="rating_num"]/text()')[0]
    evaluators_num = re.match('^\d+', info.xpath('./div[@class="bd"]/div[@class="star"]/span[4]/text()')[0]).group()
    f.write(','.join([title, brief, rating_num, evaluators_num]) + '\n')
    
f.close()

多页

In [16]:
import requests
from lxml import etree

In [17]:
def crawl_single_page(start_entry_num:int=0):
    url = f'https://movie.douban.com/top250?start={start_entry_num}'
    response = requests.get(url, headers=headers)
    tree = etree.HTML(response.content.decode('utf-8'))
    
    for info in tree.xpath('//div[@class="info"]'):
        title = info.xpath('./div[@class="hd"]/a/span[@class="title"]/text()')[0]
        brief = ' '.join([line.strip().replace('\xa0', ' ') for line in info.xpath('./div[@class="bd"]/p/text()')]).strip()
        rating_num = info.xpath('./div[@class="bd"]/div[@class="star"]/span[@class="rating_num"]/text()')[0]
        evaluators_num = re.match('^\d+', info.xpath('./div[@class="bd"]/div[@class="star"]/span[4]/text()')[0]).group()
        yield title, brief, rating_num, evaluators_num

In [18]:
max_entries_num = int(re.search('\d+', tree.xpath('//div[@class="paginator"]/span[@class="count"]/text()')[0]).group())

f = open('豆瓣top250_2.csv', 'w', encoding='utf-8')

for start in range(0, max_entries_num, 25):
    for info in crawl_single_page(start):
        f.write(','.join(info) + '\n')
    
f.close()